Rental discovery

In [79]:
import pandas as pd
import math

In [65]:
def get_taxable_income(income: int):
  """
  returns tax calculated
  based on 2022-23 tax
  """
  threshold_1 = 18200
  threshold_2 = 45000
  threshold_3 = 120000
  threshold_4 = 180000

  if income < threshold_1:
    return 0
  elif income > threshold_1 and income <= threshold_2:
    return (income - threshold_1)*0.19
  elif income > threshold_2 and income <= threshold_3:
    return 5092 + (income - threshold_2)*0.325
  elif income > threshold_3 and income <= threshold_4:
    return 29467 + (income - threshold_3)*0.37
  elif income > threshold_4:
    return 51667 + (income - threshold_4)*0.45
  


In [72]:
def generate_income(
  income=100000
):
  INCOME = income
  TAX = get_taxable_income(INCOME)
  NET_INCOME = INCOME - TAX
  TAX_PERCENTAGE = TAX/INCOME

  data = {
    "income": INCOME,
    "tax": TAX,
    "net_income": NET_INCOME,
    "tax_percentage": TAX_PERCENTAGE
  }

  return pd.Series(data), data


In [76]:
k_100, _ = generate_income(income=100000)
k_140, _ = generate_income(income=140000)
k_200, _ = generate_income(income=200000)
k_250, _ = generate_income(income=250000)
income_comparison = pd.DataFrame({
  "100k": k_100,
  "140k": k_140,
  "200k": k_200,
  "250k": k_250,
})

income_comparison.round(2)

,100k,140k,200k,250k
income,100000.00,140000.00,200000.0,250000.00
tax,22967.00,36867.00,60667.0,83167.00
net_income,77033.00,103133.00,139333.0,166833.00
tax_percentage,0.23,0.26,0.3,0.33


In [57]:
def generate_profile(
  price=600000,
  interest=0.05, 
  strata_fee=2400,
  water_bill=265,
  rent_yield=650,
  down_payment_percent=0.05
):
  # interest rates
  INTEREST_RATE = interest # adjust

  # property related
  PRICE = price # adjust
  STRATA_FEE_QUARTER = strata_fee
  STRATA_FEE_MONTH = STRATA_FEE_QUARTER/3
  WATER_BILL_QUARTER = water_bill # adjust
  WATER_BILL_MONTH = WATER_BILL_QUARTER/3
  RENT_YIELD_WEEK = rent_yield # adjust
  RENT_YIELD_MONTH = RENT_YIELD_WEEK*4

  # capital related
  DOWN_PAYMENT_PERCENT = down_payment_percent # adjust
  DOWN_PAYMENT_VALUE = DOWN_PAYMENT_PERCENT*PRICE
  BORROWED_AMOUNT = PRICE - DOWN_PAYMENT_VALUE

  YEARLY_PAYMENT = BORROWED_AMOUNT*INTEREST_RATE
  WEEKLY_PAYMENT = YEARLY_PAYMENT/52
  MONTHLY_PAYMENT = WEEKLY_PAYMENT*4

  NET_INCOME_MONTH = RENT_YIELD_MONTH - MONTHLY_PAYMENT - STRATA_FEE_MONTH - WATER_BILL_MONTH
  NET_INCOME_YEAR = NET_INCOME_MONTH * 12

  property_profile = {
    'price': PRICE,
    'down_payment': DOWN_PAYMENT_PERCENT,
    'interest': INTEREST_RATE,
    'repayment(month)': MONTHLY_PAYMENT,
    'strata_fee(month)': STRATA_FEE_MONTH,
    'water_bill(month)': WATER_BILL_MONTH,
    'rent_yield(month)': RENT_YIELD_MONTH,
    'net_income(month)': NET_INCOME_MONTH,
    'net_income(year)': NET_INCOME_YEAR,
  }
  return pd.Series(property_profile), property_profile


In [125]:
def generate_down_matrix(price=600000, rent_yield=650):
  down_5,_ = generate_profile(price=price, rent_yield=rent_yield)
  down_10,_ = generate_profile(price=price, down_payment_percent=0.1, rent_yield=rent_yield)
  down_20,_ = generate_profile(price=price, down_payment_percent=0.2, rent_yield=rent_yield)
  comparison = pd.DataFrame({
    "5% down": down_5,
    "10% down": down_10,
    "20% down": down_20,
  })
  return comparison, down_5, down_10, down_20


In [126]:
def generate_tax_benefit(income_series, property_series):
  INCOME = income_series["income"]
  TAX = income_series["tax"]
  TAX_OFFSET = abs(property_series["net_income(year)"]) if property_series["net_income(year)"] < 0 else 0
  POST_OFFSET_INCOME = INCOME - TAX_OFFSET

  POST_OFFSET_TAX = get_taxable_income(POST_OFFSET_INCOME)
  TAX_BENEFIT = TAX - POST_OFFSET_TAX

  TOTAL_PROPERTY_YIELD = property_series["net_income(year)"] + TAX_BENEFIT

  return pd.Series({
    "income": INCOME,
    "tax": TAX,
    "offset": TAX_OFFSET,
    "post_offset_income": POST_OFFSET_INCOME,
    "post_offset_tax": POST_OFFSET_TAX,
    "tax_benefit": TAX_BENEFIT,
    "total_property_yield": TOTAL_PROPERTY_YIELD
  })

In [127]:
def generate_down_benefit_matrix(price=600000):
  _, down_5, down_10, down_20 = generate_down_matrix(price)
  down_5_benefit_matrix = pd.DataFrame({
    "k100_d5": generate_tax_benefit(k_100, down_5),
    "k140_d5": generate_tax_benefit(k_140, down_5),
    "k200_d5": generate_tax_benefit(k_200, down_5),
    "k250_d5": generate_tax_benefit(k_250, down_5),
  })
  down_5_benefit_matrix.round(0)
  down_10_benefit_matrix = pd.DataFrame({
    "k100_d10": generate_tax_benefit(k_100, down_10),
    "k140_d10": generate_tax_benefit(k_140, down_10),
    "k200_d10": generate_tax_benefit(k_200, down_10),
    "k250_d10": generate_tax_benefit(k_250, down_10),
  })
  down_10_benefit_matrix.round(0)
  down_20_benefit_matrix = pd.DataFrame({
    "k100_d20": generate_tax_benefit(k_100, down_20),
    "k140_d20": generate_tax_benefit(k_140, down_20),
    "k200_d20": generate_tax_benefit(k_200, down_20),
    "k250_d20": generate_tax_benefit(k_250, down_20),
  })
  down_20_benefit_matrix.round(0)

  return down_5_benefit_matrix, down_10_benefit_matrix, down_20_benefit_matrix

In [130]:
five, ten, twenty = generate_down_benefit_matrix(600000)
df = pd.concat([five.T, ten.T, twenty.T])
df = df.sort_values(by=["tax_benefit"], ascending=False)
df.round(0)

,income,tax,offset,post_offset_income,post_offset_tax,tax_benefit,total_property_yield
k200_d5,200000.0,60667.0,5768.0,194232.0,58072.0,2595.0,-3172.0
k250_d5,250000.0,83167.0,5768.0,244232.0,80572.0,2595.0,-3172.0
k140_d5,140000.0,36867.0,5768.0,134232.0,34733.0,2134.0,-3634.0
k200_d10,200000.0,60667.0,4383.0,195617.0,58695.0,1972.0,-2411.0
k250_d10,250000.0,83167.0,4383.0,245617.0,81195.0,1972.0,-2411.0
k100_d5,100000.0,22967.0,5768.0,94232.0,21092.0,1874.0,-3893.0
k140_d10,140000.0,36867.0,4383.0,135617.0,35245.0,1622.0,-2761.0
k100_d10,100000.0,22967.0,4383.0,95617.0,21542.0,1424.0,-2959.0
k200_d20,200000.0,60667.0,1614.0,198386.0,59941.0,726.0,-888.0
k250_d20,250000.0,83167.0,1614.0,248386.0,82441.0,726.0,-888.0


# Cheaper property

In [131]:
five, ten, twenty = generate_down_benefit_matrix(price=500000)
df = pd.concat([five.T, ten.T, twenty.T])
df = df.sort_values(by=["tax_benefit"], ascending=False)
df.round(0)

,income,tax,offset,post_offset_income,post_offset_tax,tax_benefit,total_property_yield
k200_d5,200000.0,60667.0,1383.0,198617.0,60045.0,622.0,-761.0
k250_d5,250000.0,83167.0,1383.0,248617.0,82545.0,622.0,-761.0
k140_d5,140000.0,36867.0,1383.0,138617.0,36355.0,512.0,-871.0
k100_d5,100000.0,22967.0,1383.0,98617.0,22518.0,450.0,-934.0
k200_d10,200000.0,60667.0,229.0,199771.0,60564.0,103.0,-126.0
k250_d10,250000.0,83167.0,229.0,249771.0,83064.0,103.0,-126.0
k140_d10,140000.0,36867.0,229.0,139771.0,36782.0,85.0,-144.0
k100_d10,100000.0,22967.0,229.0,99771.0,22892.0,74.0,-155.0
k100_d20,100000.0,22967.0,0.0,100000.0,22967.0,0.0,2078.0
k140_d20,140000.0,36867.0,0.0,140000.0,36867.0,0.0,2078.0


In [133]:
five, ten, twenty = generate_down_benefit_matrix(price=400000)
df = pd.concat([five.T, ten.T, twenty.T])
df = df.sort_values(by=["total_property_yield"], ascending=False)
df.round(0)

,income,tax,offset,post_offset_income,post_offset_tax,tax_benefit,total_property_yield
k100_d20,100000.0,22967.0,0.0,100000.0,22967.0,0.0,5771.0
k140_d20,140000.0,36867.0,0.0,140000.0,36867.0,0.0,5771.0
k200_d20,200000.0,60667.0,0.0,200000.0,60667.0,0.0,5771.0
k250_d20,250000.0,83167.0,0.0,250000.0,83167.0,0.0,5771.0
k100_d10,100000.0,22967.0,0.0,100000.0,22967.0,0.0,3925.0
k140_d10,140000.0,36867.0,0.0,140000.0,36867.0,0.0,3925.0
k200_d10,200000.0,60667.0,0.0,200000.0,60667.0,0.0,3925.0
k250_d10,250000.0,83167.0,0.0,250000.0,83167.0,0.0,3925.0
k100_d5,100000.0,22967.0,0.0,100000.0,22967.0,0.0,3002.0
k140_d5,140000.0,36867.0,0.0,140000.0,36867.0,0.0,3002.0
